In [10]:
import pypylon.pylon as py
import numpy as np
import matplotlib.pyplot as plt
import traceback
import time
from datetime import datetime
import cv2

In [11]:
width = 1080
height = 1920
fps = 30

In [12]:
def initialize_cam(cam):
    cam.UserSetSelector = 'Default'
    cam.UserSetLoad.Execute()

    cam.ExposureAuto = 'Off'
    cam.PixelFormat = 'YCbCr422_8'
    cam.ExposureTime = 30000
    cam.AcquisitionFrameRate = fps



In [13]:
tlf = py.TlFactory.GetInstance()
cam = py.InstantCamera(tlf.CreateFirstDevice())
cam.Open()
initialize_cam(cam)

In [14]:
converter = py.ImageFormatConverter()
converter.OutputPixelFormat = py.PixelType_RGB8packed
converter.OutputBitAlignment = "MsbAligned"

In [6]:
def draw_time_date(frame):

    font = cv2.FONT_HERSHEY_SIMPLEX
    org = (1150, 50)
    fontScale = 1
    color = (255, 255, 0)
    thickness = 2

    now = datetime.now()
    dt_string = now.strftime("%d/%m/%Y %H:%M:%S")

    frame = cv2.putText(frame, dt_string, org, font, fontScale, color, thickness, cv2.LINE_AA)

    # Attempt to display using cv2 (doesn't work)
    cv2.namedWindow("Input")
    cv2.imshow("Input", frame)
    cv2.waitKey(1)
    return frame

In [7]:
output_video = cv2.VideoWriter("output.avi", cv2.VideoWriter_fourcc(*'MJPG'), fps, (width,height))

def save_video(frame):
    frame = np.frombuffer(frame, dtype=np.uint8).reshape(width, height, -1)
    frame = cv2.cvtColor(cv2.resize(frame, (height,width)), cv2.COLOR_BGR2RGB)

    frame = draw_time_date(frame)
    # output_video.write(frame)


    return

In [8]:
class ImageHandler (py.ImageEventHandler):
    def __init__(self, *args):
        super().__init__(*args)
        self.time_old = time.time()
    
    def OnImageGrabbed(self, camera, grabResult):
        try:
            if grabResult.GrabSucceeded():
                
                if (~converter.ImageHasDestinationFormat(grabResult)):
                    grabResult = converter.Convert(grabResult)
                    
                img = grabResult.Array
                time_new = time.time()
                rate = 1/(time_new-self.time_old)
                print(rate)
                self.time_old = time_new
                # img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB )
                # cv2.namedWindow("Input")
                # cv2.imshow("Input", img)
                # cv2.waitKey(1)
                save_video(img)
            else:
                raise RuntimeError("Grab failed")
        except Exception as e:
            traceback.print_exc()

In [15]:
def BackgroundLoopSample():
    handler = ImageHandler()

    cam.RegisterImageEventHandler(handler, py.RegistrationMode_ReplaceAll, py.Cleanup_None)

    # cam.StartGrabbingMax(100, py.GrabStrategy_LatestImages, py.GrabLoop_ProvidedByInstantCamera)
    cam.StartGrabbing(py.GrabStrategy_LatestImages, py.GrabLoop_ProvidedByInstantCamera)

    try:
        while cam.IsGrabbing():
            pass
    except KeyboardInterrupt:
        pass

    cam.StopGrabbing()
    cam.DeregisterImageEventHandler(handler)
    cv2.destroyAllWindows()

    # return handler.img_sum

In [16]:
BackgroundLoopSample()

6.765170125083672
2.7995079523891206
32.34199528090927
37.09705208600515
33.455937719354225
30.091501955016682
21.332682311534274
29.49249029645047
30.383819651705252
33.41835246874726
27.29067603617672
30.38668125276206
30.382499094530967
34.37785026965887
31.51597850997483
33.05282236774707
31.33327855013783
29.286355670067103
30.473444833548875
64.00781345379075
32.00805866955639
25.59703159423651
31.341941654710666
34.618179417129554
47.32161473023896
31.97194844001311
54.57922132000833
32.00854720422476
26.352586374802872
28.649226103468532
31.33070395601769
34.57537363262413
35.81263341245582
40.095442031202204
35.80285104566795
32.35746466703697
37.137453515140784
40.1034927859103
41.78134619024376
41.771775719549844
43.59484882186029
34.649637750002064
35.81018731963868
33.41861873346719
38.56760335442107
38.564411875580404
40.1034927859103
27.100414165627484
35.8120218579235
27.852103697407564
32.338504240555125
29.494356817878163
27.852843520068
31.331874173620086
30.38624097

In [17]:
cam.IsOpen()

True

In [11]:
cam.ResultingFrameRate()

30.00030000300003

In [18]:
cam.Close()